In [1]:
library(tidyverse)

# Custom package
library(rutils)

-- Attaching packages ------------------------------------------------------------------------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.3     v purrr   0.3.4
v tibble  3.0.6     v dplyr   1.0.4
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1

-- Conflicts ---------------------------------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



In [2]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
projects <- c("TCGA-CESC", "TCGA-UCS", "TCGA-UCEC", "TCGA-OV")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")
matrisome_list <- paste0(dirs$data_dir, "/matrisome/matrisome_hs_masterlist.tsv")

In [3]:
dset_idx <- 2
save_lists <- FALSE

In [4]:
p_thresh <- 0.05
lfc_thresh <- log2(2)
q_deg_thresh <- 0.05
q_anova_thresh <- 0.05
q_me_thresh <- 0.05
coxph_coeff_thresh <- 0.0
mi_thresh <- 0.0
consensus_thresh <- 0.0
consensus_n <- 5
hub_con_thresh <- 0

# Functions

In [5]:
get_consensus_col <- function(df, n, thresh = 0) {
    consensus_df <- df %>%
        dplyr::select(geneID, contains("mean")) %>%
        dplyr::mutate(consensus = rowSums(.[-1] > thresh) >= n)
    df %>% dplyr::mutate(consensus = consensus_df$consensus)
}

# Data

In [6]:
matrisome_df <- rutils::load_matrisome_df(matrisome_list) %>%
    dplyr::select(gene_symbol, division, category)
norm_counts_df <- read_tsv(paste0(dirs$data_dir, "/", unified_dsets[dset_idx], "/", "norm_counts.tsv"))
m_norm_counts_df <- norm_counts_df %>%
    dplyr::filter(geneID %in% matrisome_df$gene_symbol)
deseq_results_df <- read_tsv(paste0(dirs$analysis_dir, "/deg/", unified_dsets[dset_idx], "_DESeq_results.tsv")) %>%
    rename(base_mean = baseMean, lfc = log2FoldChange)
welch_anova_results_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_welch_anova_results.tsv"))
mi_figo_results_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_MI_figo_results.tsv")) %>%
    dplyr::rename(mi_est = MI_est_median)
cls_baselines_df <- read_tsv(paste0(dirs$analysis_dir, "/meta/", "cls_baselines.tsv"))
f1_l1_lr_scores_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_l1_lr_ref_scores.tsv"))
f1_l1_lr_results_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_l1_lr_results.tsv")) %>%
    get_consensus_col(n = consensus_n, thresh = consensus_thresh)
network_mm_gs_df <- read_tsv(paste0(dirs$analysis_dir, "/network/", unified_dsets[dset_idx], "_gene_mm_gs.tsv"))
network_me_sig_df <- read_tsv(paste0(dirs$analysis_dir, "/network/", unified_dsets[dset_idx], "_eigengene_traits.tsv"))
lnames <- load(file = paste0(dirs$data_dir, "/saved_network_objects/", unified_dsets[dset_idx], "_tumor_data.RData"))
lnames <- load(file = paste0(dirs$data_dir, "/saved_network_objects/", unified_dsets[dset_idx], "_tumor_network.RData"))


-- Column specification ------------------------------------------------------------------------------------------------------------------------
cols(
  Division = col_character(),
  Category = col_character(),
  `Gene Symbol` = col_character(),
  `Gene Name` = col_character(),
  Synonyms = col_character(),
  HGNC_IDs = col_double(),
  `HGNC_IDs Links` = col_double(),
  UniProt_IDs = col_character(),
  Refseq_IDs = col_character(),
  Orthology = col_character(),
  Notes = col_character()
)



-- Column specification ------------------------------------------------------------------------------------------------------------------------
cols(
  .default = col_double(),
  geneID = col_character()
)
i Use `spec()` for the full column specifications.



-- Column specification ------------------------------------------------------------------------------------------------------------------------
cols(
  geneID = col_character(),
  baseMean = col_double(),
  log2FoldChange = col_double(),
 

# DGE analysis

## DEG

In [7]:
deg_meta_ls <- deg_meta(deseq_results_df, lfc_thresh, q_deg_thresh, nrow(norm_counts_df))
deg_meta_ls[1:4]

$n_deg
[1] 7646

$deg_prop
[1] 0.3777295

$n_up
[1] 4682

$n_down
[1] 2964

## DEMG

In [8]:
m_deseq_results_df <- deseq_results_df %>%
    dplyr::filter(geneID %in% matrisome_df$gene_symbol)
demg_meta_ls <- deg_meta(m_deseq_results_df, lfc_thresh, q_deg_thresh, nrow(m_norm_counts_df))
demg_meta_ls[1:4]

$n_deg
[1] 595

$deg_prop
[1] 0.5902778

$n_up
[1] 363

$n_down
[1] 232

# Uni/multivariate analysis

## Welch ANOVA

In [9]:
anova_meta_ls <- simple_test_meta(welch_anova_results_df, q_anova_thresh)
anova_meta_ls[1]

$n_sig
[1] 12

## MI

In [10]:
mi_meta_ls <- mi_meta(mi_figo_results_df, 50)
mi_meta_ls[1]

$n_mi
[1] 30

# ML baselines

In [11]:
cls_baselines <- as.list(cls_baselines_df[dset_idx,])
names(cls_baselines) <- c("dataset", "naive", "mc", "n_obs")
f1_macro_majority_baseline <- cls_baselines$naive
f1_macro_mc_baseline <- cls_baselines$mc
cls_baselines

$dataset
[1] "unified_uterine_data"

$naive
[1] 0.140625

$mc
[1] 0.2460084

$n_obs
[1] 46

# LR (L1)

In [12]:
lr_l1_meta_ls <- lr_l1_meta(f1_l1_lr_scores_df, f1_l1_lr_results_df, cls_baselines)
lr_l1_meta_ls[1:4]

$avg_score
[1] 0.3045635

$naive_pct_imp
[1] 116.5785

$mc_pct_imp
[1] 23.80207

$n_genes
[1] 28

## How do uni/multivariate analyses overlap?

In [22]:
length(intersect(mi_meta_ls$genes, lr_l1_meta_ls$genes))
length(intersect(anova_meta_ls$genes, lr_l1_meta_ls$genes))
length(intersect(anova_meta_ls$genes, mi_meta_ls$genes))

[1] 0

[1] 3

[1] 2

# WGCNA

In [14]:
hub_df <- get_most_conn_genes(data_expr, module_colors, soft_power, conn_vs_hub_thresh = hub_con_thresh) %>%
    bind_rows(.id = "module") %>%
    dplyr::select(geneID, everything())

In [15]:
wgcna_meta_ls <- wgcna_meta(network_me_sig_df, network_mm_gs_df, q_me_thresh, p_thresh, hub_df$geneID)
wgcna_meta_ls[1:2]

$n_sig_modules
[1] 3

$n_sig_genes
[1] 175

## TOM meta

In [16]:
colnames(tom) <- colnames(data_expr)
rownames(tom) <- colnames(data_expr)
rowSums(tom) %>%
    quantile() %>%
    round(digits = 1)

0%  25%  50%  75% 100% 
 1.4  2.4  3.4  5.3 18.3

# Create Lists

In [17]:
deg_list <- deg_meta_ls$genes
demg_list <- demg_meta_ls$genes
umsmg_list <- lr_l1_meta_ls$genes %>%
    union(anova_meta_ls$genes) %>%
    union(mi_meta_ls$genes)
nsmg_list <- wgcna_meta_ls$genes
umsmg_demg_list <- umsmg_list %>%
    intersect(demg_list)
nsmg_demg_list <- nsmg_list %>%
    intersect(demg_list)
umsmg_nsmg_demg_list <- intersect(umsmg_demg_list, nsmg_demg_list)

In [18]:
length(deg_list)
length(demg_list)
length(umsmg_list)
length(nsmg_list)
length(umsmg_demg_list)
length(nsmg_demg_list)
length(umsmg_nsmg_demg_list)

[1] 7646

[1] 595

[1] 65

[1] 175

[1] 37

[1] 106

[1] 9

# Save lists

In [19]:
if (save_lists) {
    write_lines(deg_list, paste0(dirs$analysis_dir, "/gene_lists/", unified_dsets[dset_idx], "_deg_list.txt"))
    write_lines(demg_list, paste0(dirs$analysis_dir, "/gene_lists/", unified_dsets[dset_idx], "_demg_list.txt"))
    write_lines(umsmg_list, paste0(dirs$analysis_dir, "/gene_lists/", unified_dsets[dset_idx], "_umsmg_list.txt"))
    write_lines(nsmg_list, paste0(dirs$analysis_dir, "/gene_lists/", unified_dsets[dset_idx], "_nsmg_hub_thresh_", hub_con_thresh, "_list.txt"))
    write_lines(umsmg_demg_list, paste0(dirs$analysis_dir, "/gene_lists/", unified_dsets[dset_idx], "_umsmg_demg_list.txt"))
    write_lines(nsmg_demg_list, paste0(dirs$analysis_dir, "/gene_lists/", unified_dsets[dset_idx], "_nsmg_hub_thresh_", hub_con_thresh, "_demg_list.txt"))
    write_lines(umsmg_nsmg_demg_list, paste0(dirs$analysis_dir, "/gene_lists/", unified_dsets[dset_idx], "_umsmg_nsmg_hub_thresh_", hub_con_thresh, "_demg_list.txt"))
}